In [1]:
import copy
import random

import numpy

from functools import partial

from deap import algorithms
from deap import base
from deap import creator
from deap import tools
from deap import gp

def progn(*args):
    for arg in args:
        arg()

def prog2(out1, out2): 
    return partial(progn,out1,out2)

def prog3(out1, out2, out3):     
    return partial(progn,out1,out2,out3)

def if_then_else(condition, out1, out2):
    out1() if condition() else out2()

class AntSimulator(object):
    direction = ["north","east","south","west"]
    dir_row = [1, 0, -1, 0]
    dir_col = [0, 1, 0, -1]
    
    def __init__(self, max_moves):
        self.max_moves = max_moves
        self.moves = 0
        self.eaten = 0
        self.routine = None
        
    def _reset(self):
        self.row = self.row_start 
        self.col = self.col_start 
        self.dir = 1
        self.moves = 0  
        self.eaten = 0
        self.matrix_exc = copy.deepcopy(self.matrix)

    @property
    def position(self):
        return (self.row, self.col, self.direction[self.dir])
            
    def turn_left(self): 
        if self.moves < self.max_moves:
            self.moves += 1
            self.dir = (self.dir - 1) % 4

    def turn_right(self):
        if self.moves < self.max_moves:
            self.moves += 1    
            self.dir = (self.dir + 1) % 4
        
    def move_forward(self):
        if self.moves < self.max_moves:
            self.moves += 1
            self.row = (self.row + self.dir_row[self.dir]) % self.matrix_row
            self.col = (self.col + self.dir_col[self.dir]) % self.matrix_col
            if self.matrix_exc[self.row][self.col] == "food":
                self.eaten += 1
            self.matrix_exc[self.row][self.col] = "passed"

    def sense_food(self):
        ahead_row = (self.row + self.dir_row[self.dir]) % self.matrix_row
        ahead_col = (self.col + self.dir_col[self.dir]) % self.matrix_col        
        return self.matrix_exc[ahead_row][ahead_col] == "food"
   
    def if_food_ahead(self, out1, out2):
        return partial(if_then_else, self.sense_food, out1, out2)
   
    def run(self,routine):
        self._reset()
        while self.moves < self.max_moves:
            routine()
    
    def parse_matrix(self, matrix):
        self.matrix = list()
        for i, line in enumerate(matrix):
            self.matrix.append(list())
            for j, col in enumerate(line):
                if col == "#":
                    self.matrix[-1].append("food")
                elif col == ".":
                    self.matrix[-1].append("empty")
                elif col == "S":
                    self.matrix[-1].append("empty")
                    self.row_start = self.row = i
                    self.col_start = self.col = j
                    self.dir = 1
        self.matrix_row = len(self.matrix)
        self.matrix_col = len(self.matrix[0])
        self.matrix_exc = copy.deepcopy(self.matrix)

ant = AntSimulator(600)

pset = gp.PrimitiveSet("MAIN", 0)
pset.addPrimitive(ant.if_food_ahead, 2)
pset.addPrimitive(prog2, 2)
pset.addPrimitive(prog3, 3)
pset.addTerminal(ant.move_forward)
pset.addTerminal(ant.turn_left)
pset.addTerminal(ant.turn_right)

creator.create("FitnessMax", base.Fitness, weights=(1.0,))
creator.create("Individual", gp.PrimitiveTree, fitness=creator.FitnessMax)

toolbox = base.Toolbox()

# Attribute generator
toolbox.register("expr_init", gp.genFull, pset=pset, min_=1, max_=2)

# Structure initializers
toolbox.register("individual", tools.initIterate, creator.Individual, toolbox.expr_init)
toolbox.register("population", tools.initRepeat, list, toolbox.individual)

def evalArtificialAnt(individual):
    # Transform the tree expression to functionnal Python code
    routine = gp.compile(individual, pset)
    # Run the generated routine
    ant.run(routine)
    return ant.eaten,

toolbox.register("evaluate", evalArtificialAnt)
toolbox.register("select", tools.selTournament, tournsize=7)
toolbox.register("mate", gp.cxOnePoint)
toolbox.register("expr_mut", gp.genFull, min_=0, max_=2)
toolbox.register("mutate", gp.mutUniform, expr=toolbox.expr_mut, pset=pset)

def main():
    random.seed(69)
    
    with  open("ant_santafe_trail.txt") as trail_file:
      ant.parse_matrix(trail_file)
    
    pop = toolbox.population(n=800)
    hof = tools.HallOfFame(1)
    stats = tools.Statistics(lambda ind: ind.fitness.values)
    stats.register("avg", numpy.mean)
    stats.register("std", numpy.std)
    stats.register("min", numpy.min)
    stats.register("max", numpy.max)
    
    algorithms.eaSimple(pop, toolbox, 0.5, 0.2, 50, stats, halloffame=hof)
    
    return pop, hof, stats


In [4]:
pop, hof, stats = main()


gen	nevals	avg    	std    	min	max
0  	800   	1.62375	3.09914	0  	24 
1  	505   	5.08875	5.07699	0  	40 
2  	474   	8.61875	6.93999	0  	43 
3  	454   	12.565 	10.4698	0  	84 
4  	475   	17.8888	13.9875	0  	84 
5  	453   	24.9112	17.597 	0  	84 
6  	482   	29.8463	20.1426	0  	84 
7  	465   	34.8075	24.1085	0  	84 
8  	477   	40.2725	29.4092	0  	87 
9  	485   	44.4988	35.2752	0  	84 
10 	529   	46.6587	38.0865	0  	84 
11 	461   	52.7313	37.6647	0  	89 
12 	506   	51.315 	38.1168	0  	89 
13 	503   	49.9575	38.2683	0  	89 
14 	495   	51.1075	38.1647	0  	89 
15 	480   	53.4312	38.0338	0  	89 
16 	481   	55.1788	38.5269	0  	89 
17 	441   	56.7238	38.434 	0  	89 
18 	500   	55.6838	38.3399	0  	89 
19 	477   	56.8   	38.396 	0  	89 
20 	502   	55.075 	38.9485	0  	89 
21 	476   	58.4838	37.3875	0  	89 
22 	493   	56.8638	38.421 	0  	89 
23 	442   	61.1   	37.2785	0  	89 
24 	499   	58.1525	38.0102	0  	89 
25 	470   	62.115 	37.4193	0  	89 
26 	486   	63.23  	36.6503	0  	89 
27 	487   	64.365 	3

In [13]:
stack = []
for node in hof[0]:
    print("  " * len(stack) + str(node.name))
    if isinstance(node, gp.Primitive):
        stack.append(node.arity)
    else:
        stack[-1] -= 1
        if stack[-1] == 0:
            del stack[-1]
    
    

if_food_ahead
  if_food_ahead
    prog3
      move_forward
      move_forward
      turn_left
    turn_left
    prog3
      turn_right
      prog2
        prog2
          move_forward
          turn_right
        if_food_ahead
          move_forward
          turn_left
        turn_left
